In [1]:
import numpy as np
import pandas as pd
import os
import load
import multiprocessing as mul
from multiprocessing import Process

## IMPORTING AND SORTING THE DATA

In [2]:
####
#### IMPORTING AND SPLITTING ICDATA $$$
path = "/media/darkwake/VIB2/Project-IceCube/icecube_10year_ps/events"
filenames = ["IC40_exp.csv", "IC59_exp.csv","IC79_exp.csv", "IC86_I_exp.csv", "IC86_II_exp.csv",
"IC86_III_exp.csv", "IC86_IV_exp.csv", "IC86_V_exp.csv", "IC86_VI_exp.csv", "IC86_VII_exp.csv"]
file = filenames[0]
f = open(os.path.join(path, file), 'r')
lines = f.readlines()
column=lines[0].split()
column.pop(0)
content = []
for file in filenames:
    f = open(os.path.join(path, file), 'r')
    lines = f.readlines()
    for line in lines[1:]:
        content.append(line.split())
    f.close()
icdata = pd.DataFrame(content, columns=column)
icdata['log10(E/GeV)'] = [float(i) for i in icdata['log10(E/GeV)']]

icdata = icdata.sort_values('log10(E/GeV)')
icdata = icdata.reset_index()
icdata = icdata.drop('index', axis=1)



icpartitions = (np.log10(300), np.log10(1000), np.log10(3000), np.log10(10000))
lwall = [0]
rwall = []
for i in range(0,len(icpartitions)):
    lwall.append(min(icdata[icdata['log10(E/GeV)'] > icpartitions[i]].index))
    rwall.append(max(icdata[icdata['log10(E/GeV)'] <= icpartitions[i]].index))
lwall.pop(-1)
ic = [icdata[lwall[i]:rwall[i] + 1] for i in range(len(lwall))]
ic.pop(0)
for i in range(3):
    ic[i] = ic[i].reset_index()
    ic[i] = ic[i].rename(columns={'index': 'oindex'})


extensions = [441 - len(ic[i])%441 for i in range(3)]

icra = [[float(i) for i in ic[j]['RA[deg]']] for j in range(3)]
icdec = [[float(i) for i in ic[j]['Dec[deg]']] for j in range(3)]
icang = [[float(i) for i in ic[j]['AngErr[deg]']] for j in range(3)]

for i in range(3):
    icra[i].extend([0]*extensions[i])
    icdec[i].extend([0]*extensions[i])
    icang[i].extend([0]*extensions[i])

######################################################################################################################

#IMPORTING MSPDATA
f = open("/media/darkwake/VIB2/Project-IceCube/10milsecpsr.txt", 'r')
lines = f.readlines()

content=[]
column=lines[3].split()
for line in lines[:]:
    content.append(line.split())
    #the INITAL DATABASE IS CLUTTERED SO WE REMOVE THE NULL COLUMNS AND OTHER CLUTTER
mspdata = pd.DataFrame(content).drop(range(0,6)).dropna().drop([2,6,8,10,11,13,14], axis=1)
f.close()
line = []
lines = []

mspdata.columns = column
column = []
content=[]
mspdata.dropna(inplace=True)
mspdata = mspdata.reset_index()
mspdata = mspdata.drop(["index", "#"], axis=1)


msra = [float(i) for i in mspdata['RAJD']]
msdec = [float(i) for i in mspdata['DECJD']]

titles = ['0.3TeV - 1 TeV', '1 TeV - 3 TeV', '3 TeV - 10 TeV']


cos5 = np.cos(np.deg2rad(5))
minra = min(msra)
maxra = max(msra)
mindec = min(msdec)
maxdec = max(msdec)

,MJD[days],log10(E/GeV),AngErr[deg],RA[deg],Dec[deg],Azimuth[deg],Zenith[deg]
0,57406.38557611,1.05,14.40,36.393,40.665,310.371,130.743
1,57741.89209354,1.07,9.90,46.209,31.837,93.586,121.900
2,56247.41003029,1.11,7.35,204.188,48.589,89.178,138.519
3,57776.35241772,1.11,10.81,273.928,32.625,65.663,122.631
4,57821.93460663,1.16,12.40,159.925,35.709,74.086,125.616
...,...,...,...,...,...,...,...
1134445,57850.41489986,7.00,1.48,299.091,-19.242,135.893,70.801
1134446,56616.28975047,7.01,1.24,300.952,-48.575,312.569,41.469
1134447,57741.31320782,7.04,3.27,226.680,-7.559,64.185,82.379
1134448,58099.46879008,7.26,3.32,317.013,-34.929,22.825,55.146


The functions in the cell below drive the functions in load.py(which contains ALL the methods required for task2)

and store the outputs given by the functions from load.py in a thread queue (named $var$ in all methods) which is later converted to a list that stores all the outputs

In [7]:
def bout(icra, icdec, msra2, msdec2, extensions, var):
    cosp_ang = load.t2a2b(icra, icdec, msra2, msdec2, extensions, 'b')
    #data = load.t2bs(cosp_ang)
    #sleep(data)
    var.put(load.t2bs(cosp_ang))
    #return data
    
def cout(icra,icdec, icang, msra2, msdec2, extensions, var):
    
    #data = load.t2c(icra,icdec, icang, msra2, msdec2, extensions)
    #sleep(data)
    var.put(load.t2c(icra,icdec, icang, msra2, msdec2, extensions))
    #return data

def dout(icra, icdec, icang, msra2, msdec2, extensions, var):
    #data =  load.t2d(icra, icdec, icang, msra2, msdec2, extensions)
    #sleep(data)
    var.put(load.t2d(icra, icdec, icang, msra2, msdec2, extensions))
    #return data

## MULTITHREADING

## This cell uses multithreading to solve task 3

### 12 Threads are used!!

One thread each for the 3 methods: 2b, 2c, 2d, so total 3 threads, and 

Each iteration calculates results for 4 synthetic sets of coordinates so $3 * 4=12$ threads

The return values of load.py functions are stored in queues: var1, var2, .... var12 each dedicated to a single thread among the 12 threads.

The output values are extracted from var $i$ are stored in val $i$ and the val $i$ s are appended to output3 which is the master output array containing all outputs

In [12]:
output3=[]
for i in range(25):
    var1 = mul.Queue()
    var2 = mul.Queue()
    var3 = mul.Queue()
    var4 = mul.Queue()
    var5 = mul.Queue()
    var6 = mul.Queue()
    var7 = mul.Queue()
    var8 = mul.Queue()
    var9 = mul.Queue()
    var10 = mul.Queue()
    var11 = mul.Queue()
    var12 = mul.Queue()
    msra2, msdec2 = load.gen_cat(minra, maxra, mindec, maxdec)
    msra3, msdec3 = load.gen_cat(minra, maxra, mindec, maxdec)
    msra4, msdec4 = load.gen_cat(minra, maxra, mindec, maxdec)
    msra5, msdec5 = load.gen_cat(minra, maxra, mindec, maxdec)
    p1 = Process(target=bout, args=(icra, icdec, msra2, msdec2, extensions, var1))
    p2 = Process(target=cout, args=(icra, icdec, icang, msra2, msdec2, extensions, var2))
    p3 = Process(target=dout, args=(icra, icdec, icang, msra2, msdec2, extensions, var3))
    p4 = Process(target=bout, args=(icra, icdec, msra3, msdec3, extensions, var4))
    p5 = Process(target=cout, args=(icra, icdec, icang, msra3, msdec3, extensions, var5))
    p6 = Process(target=dout, args=(icra, icdec, icang, msra3, msdec3, extensions, var6))
    p7 = Process(target=bout, args=(icra, icdec, msra4, msdec4, extensions, var7))
    p8 = Process(target=cout, args=(icra, icdec, icang, msra4, msdec4, extensions, var8))
    p9 = Process(target=dout, args=(icra, icdec, icang, msra4, msdec4, extensions, var9))
    p10 = Process(target=bout, args=(icra, icdec, msra5, msdec5, extensions, var10))
    p11 = Process(target=cout, args=(icra, icdec, icang, msra5, msdec5, extensions, var11))
    p12 = Process(target=dout, args=(icra, icdec, icang, msra5, msdec5, extensions, var12))
    p1.start()
    p2.start()
    p3.start()
    p4.start()
    p5.start()
    p6.start()
    p7.start()
    p8.start()
    p9.start()
    p10.start()
    p11.start()
    p12.start()
    val1 = var1.get()
    val2 = var2.get()
    val3 = var3.get()
    val4 = var4.get()
    val5 = var5.get()
    val6 = var6.get()
    val7 = var7.get()
    val8 = var8.get()
    val9 = var9.get()
    val10 = var10.get()
    val11 = var11.get()
    val12 = var12.get()
    
    output3.append([val1, val2, val3, val4, val5, val6, val7, val8, val9, val10, val11, val12])

#### METHOD 2B:

    t2bbound stores the mean of Background determined from the Cosine(Space angle) distributions of all 100 sets of 441 synthetic coordinates

    t2bsignal stores the mean of corresponding Signal values

#### METHOD 2C:
    t2ccount stores the mean number of matches for each of the 100 sets of 441 synthetic coordinates
    $|cos(\theta)|  < cos X$  where \theta is the space angle and $X$ is the AngErr of Neutrino

#### METHOD 2D:
    t2dcount stores the mean number of matches for each of the 100 sets of 441 synthetic coordinates

    matches here means $|cos(\theta)-cos(5)|  < 0.5 x (1-cos X)$ where \theta is the space angle and $X$ is the AngErr of Neutrino


In [49]:
t2bbgound = [0,0,0]
t2bsignal = [0,0,0]
t2ccount = [0,0,0]
t2dcount = [0,0,0]
for i in range(25):
    for j in range(len(output3[i])):
        if j in [0, 3, 6, 9]:
            #print(output3[i][j][1])
            for k in range(3):
                t2bbgound[k] = t2bbgound[k] + output3[i][j][0][k]/25.0
                t2bsignal[k] = t2bsignal[k] + output3[i][j][1][k]/25.0
        if j in [1, 4, 7, 10]:
            for k in range(3):
                t2ccount[k] = t2ccount[k] + int(output3[i][j][k]/25.0)
        if j in [2, 5, 8, 11]:
            for k in range(3):
                t2dcount[k] = t2dcount[k] + int(output3[i][j][k]/25.0)

This cell sirts and prints the outputs in a format shown in the last markdown cell below

In [1]:
for x in range(3):
    print(f"#########  {titles[x]}  #########\nBACKGROUND: {t2bbgound[x]}\nSIGNAL: {t2bsignal[x]}\nNo.of Matches: {t2ccount[x]}\nMatches when BKGND Offset by 5 deg: {t2dcount[x]}\n\n")

NameError: name 'titles' is not defined

### FIRST SUCCESSFUL RUN ON 0202.2022-22.22 GAVE THIS OUTPUT:

#########  0.3 TeV - 1 TeV  #########

BACKGROUND: 369446.0271555127

SIGNAL: 369406.75999999995

No.of Matches: 42916

Matches when BKGND Offset by 5 deg: 123858


#########  1 TeV - 3 TeV  #########

BACKGROUND: 351063.90134929185

SIGNAL: 350299.8

No.of Matches: 31486

Matches when BKGND Offset by 5 deg: 88129


#########  3 TeV - 10 TeV  #########

BACKGROUND: 83974.27446364332

SIGNAL: 83466.76000000002

No.of Matches: 13891

Matches when BKGND Offset by 5 deg: 14407